In [1]:
include("lib/all.jl");

In [50]:
particles = make_particles(2^20);
t = Tree(particles);

In [51]:
@time group!(t, 4);

  0.173023 seconds (4 allocations: 160 bytes)


In [52]:
@time inform!(t)

  0.029635 seconds (4 allocations: 160 bytes)


In [53]:
test_in_cell_mass(t)

In [54]:
ax = zeros(length(particles))
ay = zeros(length(particles))
az = zeros(length(particles));

In [55]:
@time get_all_accel!(t, 0.5, 1.0e-157, ax,ay,az)

  3.795108 seconds (6 allocations: 256 bytes)


In [41]:
@time get_all_accel_rel!(t, 0.02, 1.0e-157, ax,ay,az)

In [ ]:
perf(particles, ax, 5000, 1.0e-157)